In [17]:
from backend.database import Database
from backend.agents import RewriterAgent, GeneratorAgent, ValidatorAgent
import pandas as pd
from time import sleep

In [18]:
database = Database()
rewriter = RewriterAgent()
generator = GeneratorAgent()
validator = ValidatorAgent()

In [33]:
eval_df = pd.read_csv(r'..\evaluation.csv')
eval_df.head()

,question,difficulty
0,List all actors' first and last names.,Easy
1,Show the titles of all films in the database.,Easy
2,Get the names of all cities.,Easy
3,List all categories available for films.,Easy
4,Show the first name and last name of all custo...,Easy


In [34]:
rewriter_responses = []
generator_responses = []
validator_responses = []

generated_queries = []
retrieved_datasets = []

In [35]:
for question in eval_df['question']:
    try:
        question = r'{}'.format(question)
        rewriter_response = rewriter.run({'question': question})
        sleep(2)
        generator_response = generator.run(rewriter_response)
        sleep(2)
        validator_response = validator.run(generator_response)
        sleep(2)
        
        print(str(validator_response) + f"\n {'-' * 30}")

        sql_query = validator_response['query']
        explanation = validator_response['explanation']
        if sql_query is not None:
            result_df = database.get_data(sql_query)
        else:
            result_df = None
        
        rewriter_responses.append(str(rewriter_response))
        generator_responses.append(str(generator_response))
        validator_responses.append(str(validator_response))
        
        generated_queries.append(sql_query)
        try: 
            retrieved_datasets.append(result_df.to_dict(orient='records'))  
        except Exception as e: 
            retrieved_datasets.append(str(e))  
            
    except Exception as e:
        print(str(e))
        generated_queries.append(str(e))
        rewriter_responses.append("")
        generator_responses.append("")
        validator_responses.append("")
        retrieved_datasets.append("")


{'question': 'Retrieve the first name and last name from the actor table for all actors.', 'query': 'SELECT first_name, last_name FROM actor;', 'explanation': "The generated SQL query is correct. It selects the first_name and last_name columns from the actor table, which directly addresses the user's request to retrieve the first and last names of all actors. The query is syntactically valid and logically sound. There are no apparent performance concerns."}
 ------------------------------
{'question': 'Retrieve the title from the film table for all films.', 'query': 'SELECT title FROM film;', 'explanation': "The query is syntactically correct and logically sound. It selects the 'title' column from the 'film' table, which directly answers the user's question. There are no apparent performance issues or edge cases to consider. The query will return all film titles present in the film table."}
 ------------------------------
{'question': "Retrieve the names from the 'city' column of the '

In [37]:
eval_df['rewriter_responses'] = rewriter_responses 
eval_df['generator_responses'] = generator_responses 
eval_df['validator_responses'] = validator_responses 
eval_df['generated_queries'] = generated_queries 
eval_df['retrieved_datasets'] = retrieved_datasets
eval_df

,question,difficulty,rewriter_responses,generator_responses,validator_responses,generated_queries,retrieved_datasets
0,List all actors' first and last names.,Easy,"{'valid': '1', 'question': 'Retrieve the first...",{'question': 'Retrieve the first name and last...,{'question': 'Retrieve the first name and last...,"SELECT first_name, last_name FROM actor;","[{'first_name': 'PENELOPE', 'last_name': 'GUIN..."
1,Show the titles of all films in the database.,Easy,"{'valid': '1', 'question': 'Retrieve the title...",{'question': 'Retrieve the title from the film...,{'question': 'Retrieve the title from the film...,SELECT title FROM film;,"[{'title': 'ACADEMY DINOSAUR'}, {'title': 'ACE..."
2,Get the names of all cities.,Easy,"{'valid': '1', 'question': ""Retrieve the names...","{'question': ""Retrieve the names from the 'cit...","{'question': ""Retrieve the names from the 'cit...",SELECT city FROM city;,"[{'city': 'A Corua (La Corua)'}, {'city': 'Abh..."
3,List all categories available for films.,Easy,"{'valid': '1', 'question': 'Show the name of e...",{'question': 'Show the name of each category f...,{'question': 'Show the name of each category f...,SELECT name FROM category;,"[{'name': 'Action'}, {'name': 'Animation'}, {'..."
4,Show the first name and last name of all custo...,Easy,"{'valid': '1', 'question': 'Retrieve the first...",{'question': 'Retrieve the first_name and last...,{'question': 'Retrieve the first_name and last...,"SELECT first_name, last_name FROM customer;","[{'first_name': 'MARY', 'last_name': 'SMITH'},..."
5,Show all films released in 2006.,Easy,"{'valid': '1', 'question': 'Show the film titl...",{'question': 'Show the film title for all film...,{'question': 'Show the film title for all film...,SELECT title FROM film WHERE release_year = 2006;,"[{'title': 'DATE SPEED'}, {'title': 'AMELIE HE..."
6,"Find all actors with the last name ""Smith.""",Easy,"{'valid': '1', 'question': ""Find the first nam...","{'question': ""Find the first name and last nam...","{'question': ""Find the first name and last nam...","SELECT first_name, last_name FROM actor WHERE ...",[]
7,"List all customers who are from the city of ""N...",Easy,"{'valid': '1', 'question': ""Retrieve the first...","{'question': ""Retrieve the first name, last na...","{'question': ""Retrieve the first name, last na...","SELECT\n c.first_name,\n c.last_name,\n c.e...",[]
8,"Get all stores located in the country ""India.""",Easy,"{'valid': '1', 'question': ""Find the store IDs...","{'question': ""Find the store IDs from the stor...","{'question': ""Find the store IDs from the stor...",SELECT DISTINCT\n s.store_id\nFROM store AS s...,"[{'store_id': 13}, {'store_id': 18}, {'store_i..."
9,Show all films with a rental rate greater than...,Easy,"{'valid': '1', 'question': 'Retrieve the film_...","{'question': 'Retrieve the film_id, title, des...","{'question': 'Retrieve the film_id, title, des...","SELECT film_id, title, description, release_ye...","[{'film_id': 2, 'title': 'ACE GOLDFINGER', 'de..."


In [39]:
eval_df.to_csv(r'..\evaluation_report.csv', index=False)